In this notebook several previously created models will be tested, namely:

* Model trained on original WikiLarge data to perform task for English
* Model trained on pairs: original english - simplified russian sentence. So, it learns both translate and simplify at the same time.
* Model trained only on the translated to Russian data.
* Model trained firstly on the original data and then on the translated corpus

All the models will be tested on the dev part of the dataset presented in RuSimpleSentEval competition. For the two of the models the russian dataset firstly will be translated into English.

## Necessary libraries

In [ ]:
! pip install googletrans
! pip install transformers
! pip install laserembeddings
! pip install sentence_transformers
! pip install language_tool_python
! pip install textstat

In [ ]:
! python -m laserembeddings download-models


✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


In [ ]:
import nltk
nltk.download('punkt')
import pandas as pd
import numpy as np
import torch
import language_tool_python
import re
import textstat
from laserembeddings import Laser
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from transformers import MarianMTModel, MarianTokenizer
from transformers.hf_api import HfApi
from googletrans import Translator
from tqdm import tqdm_notebook

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Loading data...

In [ ]:
! mkdir data
! gdown https://drive.google.com/uc?id=1bJo8TagTGKa0uyppQRqsHrKHyYO5tcZc
! gdown https://drive.google.com/uc?id=11lqipq6ggrgCk8bVxQ4-uuPVMCKN5ebU
! gdown https://drive.google.com/uc?id=1dB3X-Wx8qU_5nDG_pxAmLvo5H_sgnHrE
! gdown https://drive.google.com/uc?id=1IVz3XC8Rm7hQCyx3xCcABjhaENrKABvF

In [ ]:
test_data = pd.read_csv('/content/test_baselines.csv', sep='\t')
test_data['INPUT:source'] = test_data['INPUT:source'].apply(lambda x: re.sub(r'[\t\n\r\f\v]+', "",x))
test_data['OUTPUT:output'] = test_data['OUTPUT:output'].apply(lambda x: re.sub(r'[\t\n\r\f\v]+', "",x))
test_data.to_csv('test_data_new.csv', index=False, sep='\t')
# test_data = pd.read_csv('/content/test_data_new.csv', sep='\t')

In [ ]:
data_train = pd.read_csv('/content/wiki_train_cleaned_translated_sd.csv')
data_dev = pd.read_csv('/content/wiki_dev_cleaned_translated_sd.csv')
data_test  = pd.read_csv('/content/test_data_new.csv',  sep='\t')

In [ ]:
test_data.sample(5)

,Unnamed: 0,INPUT:source,OUTPUT:output,trunctuation_bs,gpt_bs,cos_sim_ref,cos_sim_tr_bs,cos_sim_gpt_bs
3331,9742,"Чтобы иметь возможность показать картину возможно большей зрительской аудитории, создатели фильма значительно смягчили на экране любовный контекст книги.","создатели фильма значительно смягчили на экране любовный контекст книги, чтобы показать картину возможно большей зрительской аудитории","Чтобы иметь возможность показать картину возможно большей зрительской аудитории, создатели.","Чтобы иметь возможность показать картину возможно большей зрительской аудитории, создатели фильма значительно смягчают и стимулируют интересующую аудиторию.",0.994697,0.966543,0.984472
464,1515,"В северных областях на замену дирхемам пришли западноевропейские денарии германской, английской и скандинавской чеканки.","В северных поселениях дирхемы поменяли на западноевропейские денарии германского, английского и скандинавского выпуска.",В северных областях на замену дирхемам пришли западноевропейские.,"В северных областях на замену дирхемам пришли западноевропейские денарии, такие как Польша (сокращенно - Варшава), Германия и Чехия.",0.953056,0.904038,0.936735
1812,5384,"Обозреватель издания назвал сценарий виртуозным и непредсказуемым, способным удержать на одном плане сразу две дюжины героев.",Журналист похвалил фильм.,"Обозреватель издания назвал сценарий виртуозным и непредсказуемым, способным удержать.","Обозреватель издания назвал сценарий виртуозным и непредсказуемым, способным удержать на протяжении нескольких поколений читателей.",0.964119,0.985414,0.988780
1214,3587,"Имеются данные о взаимосвязи между группами крови и частотой других инфекционных заболеваний (туберкулёз, грипп и др.).",Люди с различными группами крови по разному подвержены инфекционным заболеваниям.,Имеются данные о взаимосвязи между группами крови и частотой.,"Имеются данные о взаимосвязи между группами крови и частотой других инфекционных заболеваний, включая туберкулез.",0.951548,0.961848,0.986331
1751,5197,"Несмотря на богатую и славную историю, австрийская литература может похвастаться только одним нобелевским лауреатом.","Австрийская литература может похвастаться только одним нобелевским лауреатом, несмотря на богатую историю.","Несмотря на богатую и славную историю, австрийская литература.","Несмотря на богатую и славную историю, австрийская литература может похвастаться только одним важным историческим периодом.",0.996435,0.985989,0.994274


## Setting the environment for Fairseq mBart

In [ ]:
! wget https://dl.fbaipublicfiles.com/fairseq/models/mbart/mbart.cc25.v2.tar.gz
! tar -xzvf /content/mbart.cc25.v2.tar.gz
! apt-get install cmake build-essential pkg-config libgoogle-perftools-dev

In [ ]:
! git clone https://github.com/google/sentencepiece.git 
%cd sentencepiece
! mkdir build

In [ ]:
%cd build
!cmake ..
!make
!make install
!ldconfig -v

In [ ]:
%cd /content

/content


In [ ]:
! git clone https://github.com/pytorch/fairseq
%cd /content/fairseq/
! python -m pip install --editable .
%cd /content

! echo $PYTHONPATH

import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"

! echo $PYTHONPATH


In [ ]:
! git clone https://github.com/feralvam/easse
! git clone https://github.com/Andoree/sent_simplification.git
%cp /content/sent_simplification/sari.py /content/easse/easse

%cd easse
! pip install .

In [ ]:
%cd /content

In [ ]:
! mkdir prepared_data
! mkdir preds

## Translation to English

In [ ]:
data_test = data_test[['Unnamed: 0', 'INPUT:source','OUTPUT:output', 'trunctuation_bs', 'gpt_bs']]

In [ ]:
# helper functions

def batch_generator(
        list_of_sentences,
        size=64
):
    num_batch = len(list_of_sentences)//size
    for index in range(num_batch):
        yield list_of_sentences[index*size:(index+1)*size]
    yield list_of_sentences[num_batch*size:]


def translate_to_eng(model, tok, src, dst):

  translations_src = []
  for i in tqdm_notebook(batch_generator(src)):
    l = tok(i, return_tensors="pt", padding=True).to(device)
    translated = model.generate(**l)
    translations_src.extend([tok.decode(t, skip_special_tokens=True) for t in translated])

  translations_dst = []
  for i in tqdm_notebook(batch_generator(dst)):
    l = tok(i, return_tensors="pt", padding=True).to(device)
    translated = model.generate(**l)
    translations_dst.extend([tok.decode(t, skip_special_tokens=True) for t in translated])

  return translations_src, translations_dst


In [ ]:
# import en-ru model from transformers
model_name = 'Helsinki-NLP/opus-mt-ru-en'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
model.to(device)

In [ ]:
data_test['INPUT:source'] = data_test['INPUT:source'].astype(str)
data_test['OUTPUT:output'] = data_test['OUTPUT:output'].astype(str)

In [ ]:
# test--------------------------------------------------
src, dst = translate_to_eng(model, tokenizer, list(data_test['INPUT:source'].values), list(data_test['OUTPUT:output'].values))
for i, obj  in enumerate(src):
  src[i] = src[i].replace('&gt;', '')
  #src[i] = re.sub(r'[&gt;{1,}]', " ", src[i])
  src[i] = src[i].replace('&lt;', '')
  src[i] = src[i].replace('()', '')
  src[i] = src[i].replace('&quot;', '')
for i, obj in enumerate(dst):
  dst[i] = dst[i].replace('&gt;', '')
  dst[i] = dst[i].replace('&lt;', '')
  dst[i] = dst[i].replace('()', '')
  dst[i] = dst[i].replace('&quot;', '')
data_test['src'] = src
data_test['dst'] = dst
data_test.to_csv('/content/drive/MyDrive/MT_sentence_simpl/wiki_test_dev_eng.csv', sep='\t')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [ ]:
data_test.sample(5)

,Unnamed: 0,INPUT:source,OUTPUT:output,trunctuation_bs,gpt_bs,src,dst
302,1072,В настоящее время Австралия ведёт активную работу по установлению режима свободной торговли с Китаем — вторым после Японии внешнеторговым партнёром страны.,Австралия ведёт работу по установлению правил свободной торговли с Китаем. Это второй после Японии внешнеторговый партнёр страны.,В настоящее время Австралия ведёт активную работу по установлению режима свободной торговли.,"В настоящее время Австралия ведёт активную работу по установлению режима свободной торговли с Китаем, а также в качестве государственного департамента Австралии.","Australia is currently working actively to establish a free trade regime with China, the country's second foreign trading partner after Japan.","Australia is working to establish free trade rules with China, the country's second foreign trade partner after Japan."
547,1746,В холодное время года даже вне полярных шапок на поверхности может образовываться светлый иней.,Во прохладный период года в том числе и за пределами противоположных шапок в плоскости способен формироваться яркий изморозь.,В холодное время года даже вне полярных шапок.,В холодное время года даже вне полярных шапок на поверхности может образовываться светлый или темный мех.,"In the cold of the year, even outside the polar caps on the surface may form a light lumber.","In a cool period of the year, including beyond the opposite hats, a bright frost can form in a plane."
3061,8983,"Там нет крупных ледников и озёр, живописных долин и зубчатых гребней гор, характерных для Альп.","Там нет ледников, озер, красивых долин и зубчатых гор как в Альпах.","Там нет крупных ледников и озёр, живописных долин и.","Там нет крупных ледников и озёр, живописных долин и зубчатых берегов.","There are no major glaciers and lakes, arty valleys, and sculpted mountain crests that characterize the Alps.","There are no glaciers, no lakes, no beautiful valleys, and no mountains like the Alps."
377,1272,В период правления Хрущёва произошёл значительный поворот экономики в сторону интересов самых широких масс народа.,При Хрущеве-поворот экономики,В период правления Хрущёва произошёл значительный поворот экономики в.,В период правления Хрущёва произошёл значительный поворот экономики в сторону интересов самых широких масс населения.,"During the period of Khrushchev's reign, there was a significant shift in the economy towards the interests of the vast majority of the people.",At the Chrushchev turn of the economy
2149,6343,"Платными остались парковки на вокзалах, в аэропортах и в нескольких специально выделенных зонах городского центра.","Парковки на вокзалах, в аэропортах и в нескольких зонах города остались платными.","Платными остались парковки на вокзалах, в аэропортах и в.","Платными остались парковки на вокзалах, в аэропортах и в нескольких специальностях.","Parkings at train stations, airports and several designated areas of the city centre remained to be paid.","Parkings at train stations, airports and several areas of the city remained free of charge."


In [ ]:
#data_test = pd.read_csv('/content/drive/MyDrive/MT_sentence_simpl/wiki_test_dev_eng.csv', sep='\t')

## Inference stage

There are 4 models with the following paths:
* /content/drive/MyDrive/checkpoints_en_en/checkpoint_best.pt model for English
* /content/drive/MyDrive/checkpoints_en-ru/checkpoint_best.pt model translated with eng source and ru destination
* /content/drive/MyDrive/checkpoints_ru_ru/checkpoint_best.pt model trained only on the Russian data
* /content/drive/MyDrive/checkpoints_en_ru_ru/checkpoint_best.pt model tuned firstly on english data and then on the russian one

All the models will be evaluated according to the following aspects:

* SARI
* BLEU
* FKGL
* AWS and ASL
* Grammar aspect

Also, it is necessary to make the following change in /content/fairseq/fairseq/tasks/translation_from_pretrained_bart.py:

```
def __init__(self, args, src_dict, tgt_dict):
        super().__init__(args, src_dict, tgt_dict)
        self.args = args                  # add this line !!!!!
        self.langs = args.langs.split(",")
        for d in [src_dict, tgt_dict]:
            for l in self.langs:
```


### EN-EN model

In [ ]:
! mkdir /content/fairseq/data

In [ ]:

with open('/content/fairseq/data/train.src', "a") as f:
  for i, row in data_train.iterrows():
    f.write(row['src']+'\n')

with open('/content/fairseq/data/train.dst', "a") as f:
  for i, row in data_train.iterrows():
    f.write(row['dst']+'\n')

with open('/content/fairseq/data/dev.src', "a") as f:
  for i, row in data_dev.iterrows():
    f.write(row['src']+'\n')

with open('/content/fairseq/data/dev.dst', "a") as f:
  for i, row in data_dev.iterrows():
    f.write(row['dst']+'\n')


with open('/content/fairseq/data/test.src', "a") as f:
  for i, row in data_test.iterrows():
    f.write(row['src']+'\n')

with open('/content/fairseq/data/test.dst', "a") as f:
  for i, row in data_test.iterrows():
    f.write(row['dst']+'\n')

In [ ]:
SPM="/content/sentencepiece/build/src/spm_encode"
BPE_MODEL="/content/mbart.cc25.v2/sentence.bpe.model"
DATA_DIR="/content/fairseq/data"
SRC="src"
TGT="dst"

!$SPM --model=$BPE_MODEL < $DATA_DIR/train.$SRC > $DATA_DIR/train.spm.$SRC &
!$SPM --model=$BPE_MODEL < $DATA_DIR/train.$TGT > $DATA_DIR/train.spm.$TGT &
!$SPM --model=$BPE_MODEL < $DATA_DIR/dev.$SRC > $DATA_DIR/dev.spm.$SRC &
!$SPM --model=$BPE_MODEL < $DATA_DIR/dev.$TGT > $DATA_DIR/dev.spm.$TGT &
!$SPM --model=$BPE_MODEL < $DATA_DIR/test.$SRC > $DATA_DIR/test.spm.$SRC &
!$SPM --model=$BPE_MODEL < $DATA_DIR/test.$TGT > $DATA_DIR/test.spm.$TGT &

In [ ]:

PREPROCESSED_DATA_DIR="/content/fairseq/data"
DICT="/content/mbart.cc25.v2/dict.txt"
!fairseq-preprocess \
  --source-lang src \
  --target-lang dst \
  --trainpref /content/fairseq/data/train.spm \
  --validpref /content/fairseq/data/dev.spm \
  --testpref /content/fairseq/data/test.spm \
  --destdir /content/fairseq/data \
  --thresholdtgt 0 \
  --thresholdsrc 0 \
  --srcdict /content/mbart.cc25.v2/dict.txt \
  --tgtdict /content/mbart.cc25.v2/dict.txt \
  --workers 70

2021-04-23 00:40:48 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='/content/fairseq/data', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, source_lang='src', srcdict='/content/mbart.cc25.v2/dict.txt', suppress_crashes=False, target_lang='dst', task='translation', tensorboard_logdir=None, testpref='/content/fairseq/data

In [ ]:
! pip install sentencepiece

In [ ]:
! mkdir /content/drive/MyDrive/MT_sentence_simpl/predictions

In [ ]:
!fairseq-generate /content/fairseq/data \
  --path  /content/drive/MyDrive/checkpoints_en_en/checkpoint_best.pt \
  --task translation_from_pretrained_bart \
  --gen-subset test \
  --source-lang src --target-lang dst \
  --bpe 'sentencepiece' --sentencepiece-model /content/mbart.cc25.v2/sentence.bpe.model \
  --sacrebleu --remove-bpe 'sentencepiece' \
  --batch-size 16 --langs ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN > model_prediction.txt & 
!cat model_prediction.txt | grep -P "^H" |sort -V |cut -f 3- > model_prediction_en_en.hyp


In [ ]:
! cp /content/model_prediction_en_en.hyp /content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_en_en.hyp

### EN-RU model

In [ ]:
with open('/content/fairseq/data/train.en', "a") as f:
  for i, row in data_train.iterrows():
    f.write(row['src']+'\n')

with open('/content/fairseq/data/train.ru', "a") as f:
  for i, row in data_train.iterrows():
    f.write(row['target_y']+'\n')

with open('/content/fairseq/data/dev.en', "a") as f:
  for i, row in data_dev.iterrows():
    f.write(row['src']+'\n')

with open('/content/fairseq/data/dev.ru', "a") as f:
  for i, row in data_dev.iterrows():
    f.write(row['target_y']+'\n')


with open('/content/fairseq/data/test.en', "a") as f:
  for i, row in data_test.iterrows():
    f.write(row['src']+'\n')

with open('/content/fairseq/data/test.ru', "a") as f:
  for i, row in data_test.iterrows():
    f.write(row['OUTPUT:output']+'\n')

In [ ]:
SPM="/content/sentencepiece/build/src/spm_encode"
BPE_MODEL="/content/mbart.cc25.v2/sentence.bpe.model"
DATA_DIR="/content/fairseq/data"
SRC="en"
TGT="ru"

!$SPM --model=$BPE_MODEL < $DATA_DIR/train.$SRC > $DATA_DIR/train.spm.$SRC &
!$SPM --model=$BPE_MODEL < $DATA_DIR/train.$TGT > $DATA_DIR/train.spm.$TGT &
!$SPM --model=$BPE_MODEL < $DATA_DIR/dev.$SRC > $DATA_DIR/dev.spm.$SRC &
!$SPM --model=$BPE_MODEL < $DATA_DIR/dev.$TGT > $DATA_DIR/dev.spm.$TGT &
!$SPM --model=$BPE_MODEL < $DATA_DIR/test.$SRC > $DATA_DIR/test.spm.$SRC &
!$SPM --model=$BPE_MODEL < $DATA_DIR/test.$TGT > $DATA_DIR/test.spm.$TGT &


In [ ]:
PREPROCESSED_DATA_DIR="/content/fairseq/data"
DICT="/content/mbart.cc25.v2/dict.txt"
!fairseq-preprocess \
  --source-lang en \
  --target-lang ru \
  --trainpref /content/fairseq/data/train.spm \
  --validpref /content/fairseq/data/dev.spm \
  --testpref /content/fairseq/data/test.spm \
  --destdir /content/fairseq/data \
  --thresholdtgt 0 \
  --thresholdsrc 0 \
  --srcdict /content/mbart.cc25.v2/dict.txt \
  --tgtdict /content/mbart.cc25.v2/dict.txt \
  --workers 70


2021-04-23 01:03:55 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='/content/fairseq/data', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, source_lang='en', srcdict='/content/mbart.cc25.v2/dict.txt', suppress_crashes=False, target_lang='ru', task='translation', tensorboard_logdir=None, testpref='/content/fairseq/data/t

In [ ]:
!fairseq-generate /content/fairseq/data \
  --path  /content/drive/MyDrive/checkpoints_en-ru/checkpoint_best.pt \
  --task translation_from_pretrained_bart \
  --gen-subset test \
  --source-lang en --target-lang ru \
  --bpe 'sentencepiece' --sentencepiece-model /content/mbart.cc25.v2/sentence.bpe.model \
  --sacrebleu --remove-bpe 'sentencepiece' \
  --batch-size 16 --langs ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN > model_prediction.txt & 
!cat model_prediction.txt | grep -P "^H" |sort -V |cut -f 3- > model_prediction_en_ru.hyp


In [ ]:
! cp /content/model_prediction_en_ru.hyp /content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_en_ru.hyp

In [ ]:
! rm -r /content/fairseq/data

In [ ]:
! mkdir /content/fairseq/data

### RU-RU model

In [ ]:
with open('/content/fairseq/data/train.src', "a") as f:
  for i, row in data_train.iterrows():
    f.write(row['target_x']+'\n')

with open('/content/fairseq/data/train.dst', "a") as f:
  for i, row in data_train.iterrows():
    f.write(row['target_y']+'\n')

with open('/content/fairseq/data/dev.src', "a") as f:
  for i, row in data_dev.iterrows():
    f.write(row['target_x']+'\n')

with open('/content/fairseq/data/dev.dst', "a") as f:
  for i, row in data_dev.iterrows():
    f.write(row['target_y']+'\n')


with open('/content/fairseq/data/test.src', "a") as f:
  for i, row in data_test.iterrows():
    f.write(row['INPUT:source']+'\n')

with open('/content/fairseq/data/test.dst', "a") as f:
  for i, row in data_test.iterrows():
    f.write(row['OUTPUT:output']+'\n')

In [ ]:
data_test.head()

In [ ]:
SPM="/content/sentencepiece/build/src/spm_encode"
BPE_MODEL="/content/mbart.cc25.v2/sentence.bpe.model"
DATA_DIR="/content/fairseq/data"
SRC="src"
TGT="dst"

!$SPM --model=$BPE_MODEL < $DATA_DIR/train.$SRC > $DATA_DIR/train.spm.$SRC &
!$SPM --model=$BPE_MODEL < $DATA_DIR/train.$TGT > $DATA_DIR/train.spm.$TGT &
!$SPM --model=$BPE_MODEL < $DATA_DIR/dev.$SRC > $DATA_DIR/dev.spm.$SRC &
!$SPM --model=$BPE_MODEL < $DATA_DIR/dev.$TGT > $DATA_DIR/dev.spm.$TGT &
!$SPM --model=$BPE_MODEL < $DATA_DIR/test.$SRC > $DATA_DIR/test.spm.$SRC &
!$SPM --model=$BPE_MODEL < $DATA_DIR/test.$TGT > $DATA_DIR/test.spm.$TGT &


In [ ]:
PREPROCESSED_DATA_DIR="/content/fairseq/data"
DICT="/content/mbart.cc25.v2/dict.txt"
!fairseq-preprocess \
  --source-lang src \
  --target-lang dst \
  --trainpref /content/fairseq/data/train.spm \
  --validpref /content/fairseq/data/dev.spm \
  --testpref /content/fairseq/data/test.spm \
  --destdir /content/fairseq/data \
  --thresholdtgt 0 \
  --thresholdsrc 0 \
  --srcdict /content/mbart.cc25.v2/dict.txt \
  --tgtdict /content/mbart.cc25.v2/dict.txt \
  --workers 70


2021-04-23 02:12:23 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='/content/fairseq/data', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, source_lang='src', srcdict='/content/mbart.cc25.v2/dict.txt', suppress_crashes=False, target_lang='dst', task='translation', tensorboard_logdir=None, testpref='/content/fairseq/data

In [ ]:
!fairseq-generate /content/fairseq/data \
  --path  /content/drive/MyDrive/checkpoints_ru_ru/checkpoint_best.pt \
  --task translation_from_pretrained_bart \
  --gen-subset test \
  --source-lang src --target-lang dst \
  --bpe 'sentencepiece' --sentencepiece-model /content/mbart.cc25.v2/sentence.bpe.model \
  --sacrebleu --remove-bpe 'sentencepiece' \
  --batch-size 16 --langs ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN > model_prediction.txt & 
!cat model_prediction.txt | grep -P "^H" |sort -V |cut -f 3- > model_prediction_ru_ru.hyp


In [ ]:
! cp /content/model_prediction_ru_ru.hyp /content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_ru_ru.hyp

In [ ]:
! cp /content/model_prediction_ru_ru_very_strange.hyp /content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_ru_ru_very_strange.hyp

### EN-EN + RU-RU

In [ ]:
!fairseq-generate /content/fairseq/data \
  --path  /content/drive/MyDrive/checkpoints_en_ru_ru/checkpoint_best.pt \
  --task translation_from_pretrained_bart \
  --gen-subset test \
  --source-lang src --target-lang dst \
  --bpe 'sentencepiece' --sentencepiece-model /content/mbart.cc25.v2/sentence.bpe.model \
  --sacrebleu --remove-bpe 'sentencepiece' \
  --batch-size 16 --langs ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN > model_prediction.txt & 
!cat model_prediction.txt | grep -P "^H" |sort -V |cut -f 3- > model_prediction_en_ru_ru.hyp


In [ ]:
! cp /content/model_prediction_en_ru_ru.hyp /content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_en_ru_ru.hyp

# Evaluation

## SARI

EN-EN

In [ ]:
! python /content/sent_simplification/refs_to_easse_format.py \
--input_path /content/drive/MyDrive/MT_sentence_simpl/wiki_test_dev_eng.csv \
--output_dataset_name test_ref_data \
--src_column "src" \
--trg_column "dst" \
--output_dir /content/prepared_data

1000
3406
3401
Overall number of references: 3401


In [ ]:
! easse evaluate \
--test_set custom \
--metrics sari \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction_en_en.hyp -q

{'sari': 32.809, 'quality_estimation': {'Compression ratio': 0.98, 'Sentence splits': 1.038, 'Levenshtein similarity': 0.417, 'Exact copies': 0.0, 'Additions proportion': 0.669, 'Deletions proportion': 0.74, 'Lexical complexity score': 8.352}}


EN-RU

In [ ]:
! rm -r /content/prepared_data
! mkdir /content/prepared_data

In [ ]:
! python /content/sent_simplification/refs_to_easse_format.py \
--input_path /content/drive/MyDrive/MT_sentence_simpl/wiki_test_dev_eng.csv \
--output_dataset_name test_ref_data \
--src_column "src" \
--trg_column "OUTPUT:output" \
--output_dir /content/prepared_data

1000
3406
3406
Overall number of references: 3406


In [ ]:
! easse evaluate \
--test_set custom \
--metrics sari \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction_en_ru.hyp -q

{'sari': 39.846, 'quality_estimation': {'Compression ratio': 0.942, 'Sentence splits': 1.005, 'Levenshtein similarity': 0.15, 'Exact copies': 0.0, 'Additions proportion': 0.652, 'Deletions proportion': 0.898, 'Lexical complexity score': 10.476}}


RU-RU

In [ ]:
! rm -r /content/prepared_data
! mkdir /content/prepared_data

In [ ]:
! python /content/sent_simplification/refs_to_easse_format.py \
--input_path /content/drive/MyDrive/MT_sentence_simpl/wiki_test_dev_eng.csv \
--output_dataset_name test_ref_data \
--src_column "INPUT:source" \
--trg_column "OUTPUT:output" \
--output_dir /content/prepared_data

1000
3406
3406
Overall number of references: 3406


In [ ]:
! easse evaluate \
--test_set custom \
--metrics sari \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction_ru_ru.hyp -q

{'sari': 32.486, 'quality_estimation': {'Compression ratio': 1.0, 'Sentence splits': 1.01, 'Levenshtein similarity': 0.373, 'Exact copies': 0.002, 'Additions proportion': 0.754, 'Deletions proportion': 0.805, 'Lexical complexity score': 10.658}}


En-Ru-Ru

In [ ]:
! easse evaluate \
--test_set custom \
--metrics sari \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction_en_ru_ru.hyp -q

{'sari': 32.495, 'quality_estimation': {'Compression ratio': 0.999, 'Sentence splits': 1.01, 'Levenshtein similarity': 0.373, 'Exact copies': 0.002, 'Additions proportion': 0.753, 'Deletions proportion': 0.806, 'Lexical complexity score': 10.657}}


## BLEU

In [ ]:
with open("/content/model_prediction_en_en.hyp", 'r') as f:
  lines = [i.strip() for i in f.readlines()]

data_test['en_en'] = lines

with open("/content/model_prediction_en_ru.hyp", 'r') as f:
  lines = [i.strip() for i in f.readlines()]

data_test['en_ru'] = lines

with open("/content/model_prediction_ru_ru.hyp", 'r') as f:
  lines = [i.strip() for i in f.readlines()]

data_test['ru_ru'] = lines

with open("/content/model_prediction_en_ru_ru.hyp", 'r') as f:
  lines = [i.strip() for i in f.readlines()]

data_test['en_ru_ru'] = lines

In [ ]:
# function from easse
import numpy as np
from typing import List
import sacrebleu
import easse.utils.preprocessing as utils_prep
from collections import OrderedDict


def corpus_bleu(
    sys_sents: List[str],
    refs_sents: List[List[str]],
    smooth_method: str = 'exp',
    smooth_value: float = None,
    force: bool = True,
    lowercase: bool = False,
    tokenizer: str = '13a',
    use_effective_order: bool = False,
):

    sys_sents = [utils_prep.normalize(sent, lowercase, tokenizer) for sent in sys_sents]
    refs_sents = [[utils_prep.normalize(sent, lowercase, tokenizer) for sent in ref_sents] for ref_sents in refs_sents]

    return sacrebleu.corpus_bleu(
        sys_sents,
        refs_sents,
        smooth_method,
        smooth_value,
        force,
        lowercase=False,
        tokenize='none',
        use_effective_order=use_effective_order,
    ).score


def sentence_bleu(
    sys_sent: str,
    ref_sents: List[str],
    smooth_method: str = 'floor',
    smooth_value: float = None,
    lowercase: bool = False,
    tokenizer: str = '13a',
    use_effective_order: bool = True,
):

    return corpus_bleu(
        [sys_sent],
        [[ref] for ref in ref_sents],
        smooth_method,
        smooth_value,
        force=True,
        lowercase=lowercase,
        tokenizer=tokenizer,
        use_effective_order=use_effective_order,
    )


def corpus_averaged_sentence_bleu(
    sys_sents: List[str],
    refs_sents: List[List[str]],
    smooth_method: str = 'floor',
    smooth_value: float = None,
    lowercase: bool = False,
    tokenizer: str = '13a',
    use_effective_order: bool = True,
):

    scores = []
    for sys_sent, *ref_sents in zip(sys_sents, *refs_sents):
        scores.append(
            sentence_bleu(
                sys_sent,
                ref_sents,
                smooth_method,
                smooth_value,
                lowercase=lowercase,
                tokenizer=tokenizer,
                use_effective_order=use_effective_order,
            )
        )
    return np.mean(scores)


In [ ]:
# make lists of src, dst
output_source_path, output_references_path = 'src_bleu.txt', 'dst_bleu.txt'
src_column, trg_column = "en_en", "dst"
data_dict = OrderedDict()
for ind, row in data_test.iterrows():
        source_sentence = row[src_column]
        reference_sentence = row[trg_column]
        if data_dict.get(source_sentence) is None:
            data_dict[source_sentence] = []
        data_dict[source_sentence].append(reference_sentence)
max_num_reference_sentences = max([len(x) for x in data_dict.values()])
src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
corpus_averaged_sentence_bleu(src, dst)

35.616937541849055

In [ ]:
# make lists of src, dst
output_source_path, output_references_path = 'src_bleu.txt', 'dst_bleu.txt'
src_column, trg_column = "en_ru", "OUTPUT:output"
data_dict = OrderedDict()
for ind, row in test_data.iterrows():
        source_sentence = row[src_column]
        reference_sentence = row[trg_column]
        if data_dict.get(source_sentence) is None:
            data_dict[source_sentence] = []
        data_dict[source_sentence].append(reference_sentence)
max_num_reference_sentences = max([len(x) for x in data_dict.values()])
src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
corpus_averaged_sentence_bleu(src, dst)

24.761650580786526

In [ ]:
# make lists of src, dst
output_source_path, output_references_path = 'src_bleu.txt', 'dst_bleu.txt'
src_column, trg_column = "ru_ru", "OUTPUT:output"
data_dict = OrderedDict()
for ind, row in test_data.iterrows():
        source_sentence = row[src_column]
        reference_sentence = row[trg_column]
        if data_dict.get(source_sentence) is None:
            data_dict[source_sentence] = []
        data_dict[source_sentence].append(reference_sentence)
max_num_reference_sentences = max([len(x) for x in data_dict.values()])
src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
corpus_averaged_sentence_bleu(src, dst)

34.749903130866464

In [ ]:
# make lists of src, dst
output_source_path, output_references_path = 'src_bleu.txt', 'dst_bleu.txt'
src_column, trg_column = "en_ru_ru", "OUTPUT:output"
data_dict = OrderedDict()
for ind, row in test_data.iterrows():
        source_sentence = row[src_column]
        reference_sentence = row[trg_column]
        if data_dict.get(source_sentence) is None:
            data_dict[source_sentence] = []
        data_dict[source_sentence].append(reference_sentence)
max_num_reference_sentences = max([len(x) for x in data_dict.values()])
src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
corpus_averaged_sentence_bleu(src, dst)

34.749903130866464

## Cos Sim

En-En

In [ ]:
from transformers import RobertaTokenizer, RobertaModel, AutoConfig, AutoTokenizer, AutoModelForMaskedLM
device = "cuda" if torch.cuda.is_available() else "cpu"
config = AutoConfig.from_pretrained("roberta-base") # "roberta-base" 'xlm-mlm-100-1280' 'xlm-roberta-base' 'bert-base-multilingual-cased'
config.output_hidden_states = True

tok = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModelForMaskedLM.from_pretrained("roberta-base", config=config)
model.to(device)

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# shape should be [1, something (768, ex)]

# import numpy as np
# def cs(a, b):
#   return (a @ b.T)/(np.linalg.norm(a)*np.linalg.norm(b))

def calc_cos_sim(df, model,tok, x, y, column_name):
    Cos_sim= []
    for index, row in df.iterrows():
        
        # original
          sentence_A = tok.encode(row[x], padding='max_length', max_length=50, truncation=True, return_tensors='pt')
          sentence_A = sentence_A.to(device)
          output = model(sentence_A)
          sent_emb = output[-1][0]
          emb_source = sent_emb.mean(axis=1)
          emb_source = emb_source.cpu().detach().numpy()

          sentence_B = tok.encode(row[y], padding='max_length', max_length=50, truncation=True, return_tensors='pt')
          sentence_B = sentence_B.to(device)
          output = model(sentence_B)
          sent_emb = output[-1][0]
          emb_target= sent_emb.mean(axis=1)
          emb_target = emb_target.cpu().detach().numpy()

          cos_val = cosine_similarity(emb_source.reshape(emb_source.shape[0], -1), emb_target.reshape(emb_target.shape[0], -1))[0][0]
          Cos_sim.append(cos_val)
    df[column_name] = Cos_sim

In [ ]:
calc_cos_sim(data_test, model, tok, 'src', 'en_en', 'cos_sim_en_en')

In [ ]:
data_test.cos_sim_en_en.mean()

0.9784409027356333

En-ru

In [ ]:
from transformers import RobertaTokenizer, RobertaModel, AutoConfig, AutoTokenizer, AutoModelForMaskedLM
device = "cuda" if torch.cuda.is_available() else "cpu"
config = AutoConfig.from_pretrained("DeepPavlov/rubert-base-cased") # "roberta-base" 'xlm-mlm-100-1280' 'xlm-roberta-base' 'bert-base-multilingual-cased'
config.output_hidden_states = True

tok = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = AutoModelForMaskedLM.from_pretrained("DeepPavlov/rubert-base-cased", config=config)
model.to(device)

In [ ]:
calc_cos_sim(data_test, model, tok, 'INPUT:source', 'en_ru', 'cos_sim_en_ru')
calc_cos_sim(data_test, model, tok, 'INPUT:source', 'ru_ru', 'cos_sim_ru_ru')
calc_cos_sim(data_test, model, tok, 'INPUT:source', 'en_ru_ru', 'cos_sim_en_ru_ru')

In [ ]:
data_test.cos_sim_en_ru.mean(), data_test.cos_sim_ru_ru.mean(), data_test.cos_sim_en_ru_ru.mean()

(0.9557205252021254, 0.9955931419634637, 0.9944967627875327)

### Flesch Kincaid Grade Level

En-en

In [ ]:
textstat.set_lang('en')

In [ ]:
a = data_test['en_en'].values
en_en = sum(textstat.flesch_kincaid_grade(i) for i in a)/len(a)
en_en

11.808367586611897

In [ ]:
# a = ' '.join(data_test['en_en'].values)
# en_en = textstat.flesch_kincaid_grade(a)
# en_en

11.1

In [ ]:
a = data_test['en_en'].values
en_en = sum(textstat.syllable_count(i) for i in a)/len(a)
en_en

34.025836758661185

In [ ]:
a = data_test['en_en'].values
en_en = sum(textstat.lexicon_count(i) for i in a)/len(a)
en_en

20.862008220786848

En-ru, ru-ru, en-ru-ru

In [ ]:
textstat.set_lang('ru')

In [ ]:
a = data_test['en_ru'].values
b = data_test['ru_ru'].values
c = data_test['en_ru_ru'].values
en_ru = sum(textstat.flesch_kincaid_grade(i) for i in a)/len(a)
ru_ru = sum(textstat.flesch_kincaid_grade(i) for i in b)/len(b)
en_ru_ru = sum(textstat.flesch_kincaid_grade(i) for i in c)/len(c)
en_ru, ru_ru, en_ru_ru

(20.034380504991258, 22.588167938931395, 22.552055196711766)

In [ ]:
# a = ' '.join(data_test['en_ru'].values)
# b = ' '.join(data_test['ru_ru'].values)
# c = ' '.join(data_test['en_ru_ru'].values)
# en_ru = textstat.flesch_kincaid_grade(a)
# ru_ru = textstat.flesch_kincaid_grade(b) 
# en_ru_ru = textstat.flesch_kincaid_grade(c)
# en_ru, ru_ru, en_ru_ru


(172.4, 1146.1, 1421.4)

In [ ]:
a = data_test['en_ru'].values
b = data_test['ru_ru'].values
c = data_test['en_ru_ru'].values
en_ru = sum(textstat.syllable_count(i) for i in a)/len(a)
ru_ru = sum(textstat.syllable_count(i) for i in b)/len(b)
en_ru_ru = sum(textstat.syllable_count(i) for i in c)/len(c)
en_ru, ru_ru, en_ru_ru

(39.49412800939518, 45.09806224310041, 44.913094539048735)

In [ ]:
a = data_test['en_ru'].values
b = data_test['ru_ru'].values
c = data_test['en_ru_ru'].values
en_ru = sum(textstat.lexicon_count(i) for i in a)/len(a)
ru_ru = sum(textstat.lexicon_count(i) for i in b)/len(b)
en_ru_ru = sum(textstat.lexicon_count(i) for i in c)/len(c)
en_ru, ru_ru, en_ru_ru

(15.870522607163828, 17.010569583088667, 16.92542571931885)

In [ ]:
import requests
text = ' '.join(data_test['INPUT:source'].values)
response = requests.post("http://api.plainrussian.ru/api/1.0/ru/measure/", data={"text":text})
response.json()

{'debug': '0',
 'indexes': {'grade_SMOG': '4 - 6 курсы ВУЗа (возраст примерно: 20-22 лет)',
  'grade_ari': 'Аспирантура, второе высшее образование, phD',
  'grade_cl': '4 - 6 курсы ВУЗа (возраст примерно: 20-22 лет)',
  'grade_dc': '1 - 3 курсы ВУЗа (возраст примерно: 17-19 лет)',
  'grade_fk': '4 - 6 курсы ВУЗа (возраст примерно: 20-22 лет)',
  'index_SMOG': 14.557852197020122,
  'index_ari': 17.71203679321743,
  'index_cl': 16.38818426966293,
  'index_dc': 13.702683787810848,
  'index_fk': 17.377772067441356},
 'lang': 'ru',
 'metrics': {'avg_slen': 15.85661345496009,
  'avg_syl': 3.0165752808988766,
  'c_share': 16.981573033707864,
  'chars': 473430,
  'letters': 393678,
  'n_complex_words': 9446,
  'n_sentences': 3508,
  'n_simple_words': 46179,
  'n_syllabes': 167797,
  'n_words': 55625,
  'spaces': 60292,
  'wsyllabes': {'1': 9867,
   '10': 35,
   '11': 6,
   '12': 4,
   '2': 12788,
   '3': 13723,
   '4': 9801,
   '5': 6023,
   '6': 2491,
   '7': 677,
   '8': 173,
   '9': 37}},
 

In [ ]:
import requests
text = ' '.join(data_test['OUTPUT:output'].values)
response = requests.post("http://api.plainrussian.ru/api/1.0/ru/measure/", data={"text":text})
response.json()

{'debug': '0',
 'indexes': {'grade_SMOG': '10 - 11-й класс (возраст примерно: 15-16 лет)',
  'grade_ari': '1 - 3 курсы ВУЗа (возраст примерно: 17-19 лет)',
  'grade_cl': '1 - 3 курсы ВУЗа (возраст примерно: 17-19 лет)',
  'grade_dc': '10 - 11-й класс (возраст примерно: 15-16 лет)',
  'grade_fk': '1 - 3 курсы ВУЗа (возраст примерно: 17-19 лет)',
  'index_SMOG': 11.356979245625993,
  'index_ari': 13.911711320522606,
  'index_cl': 13.275588954554713,
  'index_dc': 10.54920179059317,
  'index_fk': 13.869039895378684},
 'lang': 'ru',
 'metrics': {'avg_slen': 12.795306388526727,
  'avg_syl': 2.8380374974526186,
  'c_share': 12.78275932341553,
  'chars': 317164,
  'letters': 259382,
  'n_complex_words': 5018,
  'n_sentences': 3068,
  'n_simple_words': 34238,
  'n_syllabes': 111410,
  'n_words': 39256,
  'spaces': 43375,
  'wsyllabes': {'1': 7690,
   '10': 11,
   '11': 1,
   '12': 1,
   '14': 1,
   '15': 1,
   '2': 10187,
   '3': 9546,
   '4': 6815,
   '5': 3294,
   '6': 1274,
   '7': 336,
   

In [ ]:
import requests
text = ' '.join(data_test['en_ru'].values)
response = requests.post("http://api.plainrussian.ru/api/1.0/ru/measure/", data={"text":text})
response.json()

{'debug': '0',
 'indexes': {'grade_SMOG': '1 - 3 курсы ВУЗа (возраст примерно: 17-19 лет)',
  'grade_ari': '4 - 6 курсы ВУЗа (возраст примерно: 20-22 лет)',
  'grade_cl': '1 - 3 курсы ВУЗа (возраст примерно: 17-19 лет)',
  'grade_dc': '1 - 3 курсы ВУЗа (возраст примерно: 17-19 лет)',
  'grade_fk': '4 - 6 курсы ВУЗа (возраст примерно: 20-22 лет)',
  'index_SMOG': 12.406806113663269,
  'index_ari': 15.059364916077413,
  'index_cl': 14.23091354870612,
  'index_dc': 11.560336482490253,
  'index_fk': 15.027498677881688},
 'lang': 'ru',
 'metrics': {'avg_slen': 13.871391826236067,
  'avg_syl': 2.8955208505027197,
  'c_share': 14.082330641173561,
  'chars': 411959,
  'letters': 327257,
  'n_complex_words': 6835,
  'n_sentences': 3499,
  'n_simple_words': 41701,
  'n_syllabes': 140537,
  'n_words': 48536,
  'spaces': 54374,
  'wsyllabes': {'1': 8966,
   '10': 4,
   '2': 12415,
   '3': 11739,
   '4': 8581,
   '5': 4441,
   '6': 1876,
   '7': 428,
   '8': 71,
   '9': 15}},
 'status': 0}

In [ ]:
import requests
text = ' '.join(data_test['en_ru_ru'].values)
response = requests.post("http://api.plainrussian.ru/api/1.0/ru/measure/", data={"text":text})
response.json()

{'debug': '0',
 'indexes': {'grade_SMOG': '1 - 3 курсы ВУЗа (возраст примерно: 17-19 лет)',
  'grade_ari': 'Аспирантура, второе высшее образование, phD',
  'grade_cl': '4 - 6 курсы ВУЗа (возраст примерно: 20-22 лет)',
  'grade_dc': '1 - 3 курсы ВУЗа (возраст примерно: 17-19 лет)',
  'grade_fk': '4 - 6 курсы ВУЗа (возраст примерно: 20-22 лет)',
  'index_SMOG': 14.281984238205567,
  'index_ari': 17.53083943300245,
  'index_cl': 16.283248101831177,
  'index_dc': 13.516749541078948,
  'index_fk': 17.17821311474296},
 'lang': 'ru',
 'metrics': {'avg_slen': 15.32686822589846,
  'avg_syl': 3.014385142176567,
  'c_share': 16.906729194580915,
  'chars': 456511,
  'letters': 380029,
  'n_complex_words': 9085,
  'n_sentences': 3506,
  'n_simple_words': 44651,
  'n_syllabes': 161981,
  'n_words': 53736,
  'spaces': 58090,
  'wsyllabes': {'1': 9523,
   '10': 35,
   '11': 6,
   '12': 4,
   '2': 12377,
   '3': 13254,
   '4': 9497,
   '5': 5829,
   '6': 2373,
   '7': 636,
   '8': 163,
   '9': 39}},
 '

In [ ]:
import requests
text = ' '.join(data_test['ru_ru'].values)
response = requests.post("http://api.plainrussian.ru/api/1.0/ru/measure/", data={"text":text})
response.json()

{'debug': '0',
 'indexes': {'grade_SMOG': '1 - 3 курсы ВУЗа (возраст примерно: 17-19 лет)',
  'grade_ari': 'Аспирантура, второе высшее образование, phD',
  'grade_cl': '4 - 6 курсы ВУЗа (возраст примерно: 20-22 лет)',
  'grade_dc': '1 - 3 курсы ВУЗа (возраст примерно: 17-19 лет)',
  'grade_fk': '4 - 6 курсы ВУЗа (возраст примерно: 20-22 лет)',
  'index_SMOG': 14.294811353101062,
  'index_ari': 17.531640673671966,
  'index_cl': 16.275580568061812,
  'index_dc': 13.505160777911009,
  'index_fk': 17.200720714980243},
 'lang': 'ru',
 'metrics': {'avg_slen': 15.412050256996002,
  'avg_syl': 3.01406258684898,
  'c_share': 16.843606988679525,
  'chars': 458434,
  'letters': 381506,
  'n_complex_words': 9091,
  'n_sentences': 3502,
  'n_simple_words': 44882,
  'n_syllabes': 162678,
  'n_words': 53973,
  'spaces': 58381,
  'wsyllabes': {'1': 9581,
   '10': 35,
   '11': 6,
   '12': 4,
   '2': 12369,
   '3': 13339,
   '4': 9593,
   '5': 5826,
   '6': 2384,
   '7': 648,
   '8': 156,
   '9': 32}},


### Grammar check

In [ ]:
tool = language_tool_python.LanguageTool('en')

Unzipping /tmp/tmpzx1ghdaj.zip to /root/.cache/language_tool_python.
Downloaded https://www.languagetool.org/download/LanguageTool-5.2.zip to /root/.cache/language_tool_python.


In [ ]:
def get_mistakes_summary(df_test, x):
    test = list(df_test[x].values)
    matches = []
    for i in test:
      matches.extend(tool.check(i))

    categories = set([i.category for i in matches])

    categories = {i:0 for i in categories}

    for i in matches:
      categories[i.category]+=1

    return categories

In [ ]:
errors = get_mistakes_summary(test_data, 'en_en')

In [ ]:
errors

{'CASING': 38,
 'COLLOCATIONS': 2,
 'GRAMMAR': 23,
 'MISC': 1,
 'PUNCTUATION': 183,
 'REDUNDANCY': 28,
 'STYLE': 3,
 'TYPOGRAPHY': 8081,
 'TYPOS': 10}

In [ ]:
tool = language_tool_python.LanguageTool('ru')

In [ ]:
errors = get_mistakes_summary(test_data, 'en_ru')

In [ ]:
errors

{'CASING': 57,
 'GRAMMAR': 163,
 'LOGIC': 12,
 'MISC': 131,
 'PUNCTUATION': 48,
 'STYLE': 27,
 'TYPOGRAPHY': 230,
 'TYPOS': 1073}

In [ ]:
errors = get_mistakes_summary(test_data, 'ru_ru')

In [ ]:
errors

{'CASING': 8,
 'GRAMMAR': 22,
 'LOGIC': 27,
 'PUNCTUATION': 20,
 'STYLE': 14,
 'TYPOGRAPHY': 393,
 'TYPOS': 800}

In [ ]:
errors = get_mistakes_summary(test_data, 'en_ru_ru')

In [ ]:
errors

{'CASING': 17,
 'GRAMMAR': 20,
 'LOGIC': 27,
 'PUNCTUATION': 21,
 'STYLE': 14,
 'TYPOGRAPHY': 398,
 'TYPOS': 800}

In [ ]:
# data_test.to_csv('/content/drive/MyDrive/MT_sentence_simpl/predictions/all_models_pred.csv', index='False', sep='\t')
# data_test = pd.read_csv('/content/drive/MyDrive/MT_sentence_simpl/predictions/all_models_pred.csv',  sep='\t')

In [ ]:
data_test[['INPUT:source', 'en_ru']].sample(6)

,INPUT:source,en_ru
2115,Перевод туризма в систему профсоюзов первоначально сопровождался отменой зачёта достижений туристов в виде спортивных разрядов и званий.,Передача туризма в систему профсоюзов вначале сопровождалась отменением туристических достижения в виде видов спорта и титулов.
1560,"На Марсе имеется множество геологических образований, напоминающих водную эрозию, в частности, высохшие русла рек.","Есть много геологических объектов на Марсе, похожих на эрозию воды, таких как сухие реки."
3171,"Украина богата туристическими возможностями: Карпаты, исторические памятники старинных украинских городов, сельский (зелёный) туризм и природные заповедники.","В Украине много туристических возможностей: Карпатины, исторические памятники древних украинских городов, сельский (зеленый) туризм и естественные заповедники."
1085,"Зависимость численности, распределения, плодовитости и миграций белки, кедровки и прочих от урожая кедровых орехов и других хвойных пород.","Это зависит от размера, распределения, плодородности и миграции белков, герцогства и других факторов на производство pine nuts и других деревьев."
712,Военное командование противоборствующих сторон в годы Гражданской войны сумело успешно решить вопросы использования крупных масс кавалерии для выполнения оперативных задач.,Военное командование вооруженных сил во время Гражданской войны успешно устранило проблему использования больших массов армии для выполнения операций.
3025,Также во многих играх серии присутствуют предметы коллекционирования и случайные события в открытом мире.,Многие игры серии также содержат коллекционные предметы и случайные события в открытом мире.


In [ ]:
data_test[['INPUT:source', 'ru_ru']].head(5)

,INPUT:source,ru_ru
0,"14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок Советский, после чего поселковый совет стал называться Советским.","14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок Советский, после чего поселковый совет стал называться Советским."
1,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.
2,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.
3,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.
4,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.
